# G2


In [17]:
import numpy as np
import pandas as pd

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

def objective_function_g2(x):
    x = np.array(x).reshape(-1, 7)
    x1, x2, x3, x4, x5, x6, x7 = x[:, 0], x[:, 1], x[:, 2], x[:, 3], x[:, 4], x[:, 5], x[:, 6]
    
    f = (x1 - 10)**2 + 5*(x2 - 12)**2 + x3**4 + 3*(x4 - 11)**2 + 10*x5**6 + 7*x6**2 + x7**4 - 4*x6*x7 - 10*x6 - 8*x7
    return f

def constraint_function_g2(x):
    x = np.array(x).reshape(-1, 7)
    x1, x2, x3, x4, x5, x6, x7 = x[:, 0], x[:, 1], x[:, 2], x[:, 3], x[:, 4], x[:, 5], x[:, 6]
    
    g1 = -127 + 2*x1**2 + 3*x2**4 + x3 + 4*x4**2 + 5*x5
    g2 = -282 + 7*x1 + 3*x2 + 10*x3**2 + x4 - x5
    g3 = -196 + 23*x1 + x2**2 + 6*x6**2 - 8*x7
    g4 = 4*x1**2 + x2**2 - 3*x1*x2 + 2*x3**2 + 5*x6 - 11*x7

    return np.vstack((g1, g2, g3, g4))

def constraints_g2(x):

    return np.all(constraint_function_g2(x) <= 0, axis=0)

def objective_function_with_constraints_g2(x):
    if not constraints_g2(x):
        return 150000
    else:
        return objective_function_g2(x)




## PSO 

In [18]:
import numpy as np
import pandas as pd

# Número de partículas
n_particles = 50

# Número de dimensões
n_dimensions = 7

# Limites inferiores e superiores para cada dimensão
lb = [-10]*7 
ub = [10]*7 

# Parâmetros do PSO
c1 = 2
c2 = 2
w = 0.7
max_iter = 300
n_experiments = 30

# Armazenar resultados
results = []
 
for exp in range(n_experiments):
    #Inicializar enxame
    particles = np.empty((n_particles, n_dimensions))
    for i in range(n_particles):
        while True:
            particle = np.random.uniform(low=lb, high=ub, size=n_dimensions)
            if objective_function_with_constraints_g2(particle) != 150000:
                particles[i] = particle
                break
    personal_best_scores = np.array([objective_function_with_constraints_g2(p) for p in particles])

    velocities = np.zeros((n_particles, n_dimensions))
    personal_best_positions = particles.copy()

    # Inicializar o melhor global
    global_best_position = personal_best_positions[np.argmin(personal_best_scores)]
    global_best_score = min(personal_best_scores)
    print('Global best score: ',global_best_score)
    # Loop principal
    for i in range(max_iter):
        for j in range(n_particles):
            # Atualizar velocidade
            velocities[j] = (w * velocities[j] +
                             c1 * np.random.rand() * (personal_best_positions[j] - particles[j]) +
                             c2 * np.random.rand() * (global_best_position - particles[j]))
            
            # Atualizar posição
            particles[j] += velocities[j]
            
            # Aplicar limites
            particles[j] = np.clip(particles[j], lb, ub)
            
            # Atualizar o melhor pessoal
            score = objective_function_with_constraints_g2(particles[j])
            if score < personal_best_scores[j]:
                personal_best_positions[j] = particles[j].copy()
                personal_best_scores[j] = score
                
                # Atualizar o melhor global
                if score < global_best_score:
                    global_best_position = particles[j].copy()
                    global_best_score = score

        # Armazenar resultados
        results.append({
            'Experiment': exp+1,
            'Iteration': i+1,
            'Best Fitness': global_best_score,
            'Best Particle': global_best_position,
            'Worst Fitness': max(personal_best_scores),
            'Average Fitness': np.mean(personal_best_scores),
            'Standard Deviation': np.std(personal_best_scores),
            'Median Fitness': np.median(personal_best_scores),
        })
        

# Criar dataframe com os resultados
df = pd.DataFrame(results)

df.head(5)

Global best score:  [880.23351066]
Global best score:  [1738.25860261]
Global best score:  [957.73088798]
Global best score:  [1546.31175982]
Global best score:  [1167.05010849]
Global best score:  [1469.31831821]
Global best score:  [1225.04199783]
Global best score:  [1378.8123671]
Global best score:  [2466.4779677]
Global best score:  [1255.57788814]
Global best score:  [1349.74945129]
Global best score:  [3078.34842727]
Global best score:  [1634.61708822]
Global best score:  [1715.80932052]
Global best score:  [1708.32589395]
Global best score:  [1635.63141338]
Global best score:  [1196.79364296]
Global best score:  [1170.84942151]
Global best score:  [1036.66650886]
Global best score:  [1238.61758923]
Global best score:  [1372.7122436]
Global best score:  [899.17284603]
Global best score:  [1420.26460388]
Global best score:  [2662.0761638]
Global best score:  [2046.4727182]
Global best score:  [861.68282596]
Global best score:  [2186.84411712]
Global best score:  [1427.64084001]
G

,Experiment,Iteration,Best Fitness,Best Particle,Worst Fitness,Average Fitness,Standard Deviation,Median Fitness
0,1,1,[850.1191719485091],"[0.6623452262911074, 1.7384967691857183, 1.993...",[680.923941957334],140121.401257,345504.211978,73630.921648
1,1,2,[850.1191719485091],"[0.6623452262911074, 1.7384967691857183, 1.993...",[680.9211449651805],82175.580447,106164.530399,22709.721023
2,1,3,[847.5305353530287],"[0.638036523201352, 1.6928534588330837, 1.9780...",[680.9209092254216],51375.165030,65660.769225,5401.254826
3,1,4,[845.7675002846155],"[0.5909508077179442, 1.6044428218258484, 1.947...",[680.92205907993],42442.727174,61639.668401,3837.375884
4,1,5,[837.0852400517052],"[0.5828572079986827, 1.6332611356921134, 2.083...",[680.9374216448632],32813.947708,55413.385035,2791.774501


In [21]:
df_pso = df.sort_values('Best Fitness', ascending=True).reset_index()
best_PSO_individual = df_pso.loc[0, 'Best Particle']
fitness_best_PSO_individual = df_pso.loc[0, 'Best Fitness']
df_pso.head()

,index,Experiment,Iteration,Best Fitness,Best Particle,Worst Fitness,Average Fitness,Standard Deviation,Median Fitness
0,1199,4,300,[680.7384017684655],"[2.31892106209504, 1.9639805917639315, -0.4730...",[680.8573148544574],680.761159,0.027847,680.750069
1,1198,4,299,[680.7384017684655],"[2.31892106209504, 1.9639805917639315, -0.4730...",[680.8573148544574],680.771612,0.025771,680.763387
2,6299,21,300,[680.7461714847584],"[2.2928969908819754, 1.9463965160506267, -0.63...",[681.615374946513],680.815162,0.137169,680.770876
3,6298,21,299,[680.7477500294672],"[2.2869400765943255, 1.9477177406657005, -0.63...",[681.615374946513],680.816450,0.136604,680.770876
4,6297,21,298,[680.7513322314056],"[2.285914121177941, 1.9457075471456227, -0.629...",[681.615374946513],680.817448,0.136191,680.773587


## GA

In [28]:
# Definindo os parâmetros do algoritmo genético
n_particles = 50
n_dimensions = 7
generations = 500
lb = [-10]*7
ub = [10]*7
crossover_prob = 0.6
mutation_prob = 0.2


results_ga = []


# Inicialização da população
particles = np.empty((n_particles, n_dimensions))
for i in range(n_particles):
    while True:
        particle = np.random.uniform(low=lb, high=ub, size=n_dimensions)
        if objective_function_with_constraints_g2(particle) != 150000:
            particles[i] = particle
            break
population = particles

# Loop das gerações
for gen in range(generations):
    # Avaliação da população
    fitness_values = [objective_function_with_constraints_g2(ind) for ind in population]

    # Seleção por torneio
    parents = []
    for _ in range(pop_size):
        contenders_indices = np.random.choice(len(population), size=2)
        contenders = [population[i] for i in contenders_indices]
        fitness_contenders = [fitness_values[i] for i in contenders_indices]
        parents.append(contenders[np.argmin(fitness_contenders)])

    # Assegurando que o melhor indivíduo sempre estará na próxima geração
    best_individual = tuple(population[np.argmin(fitness_values)])
    if best_individual not in [tuple(parent) for parent in parents]:
        parents[0] = np.array(best_individual)

        # Crossover
    alpha = 0.5  # Este é o parâmetro alfa, que determina o tamanho do intervalo de blend
    offspring = []
    for i in range(pop_size//2):
        parent1 = parents[i * 2]
        parent2 = parents[i * 2 + 1]
        if np.random.rand() <= crossover_prob:
            # Blend Crossover
            d = abs(parent1 - parent2)
            low = np.minimum(parent1, parent2) - alpha * d
            high = np.maximum(parent1, parent2) + alpha * d
            child1 = np.random.uniform(low, high)
            child2 = np.random.uniform(low, high)
            child1, child2 = np.clip(child1, lb, ub), np.clip(child2, lb, ub)

            # Uniform Crossover
            mask = np.random.randint(0, 2, size=genes).astype(np.bool)
            child1_u, child2_u = parent1.copy(), parent2.copy()
            child1_u[mask], child2_u[mask] = parent2[mask], parent1[mask]
            child1_u, child2_u = np.clip(child1_u, lb, ub), np.clip(child2_u, lb, ub)

            # Heuristic Crossover
            r = np.random.rand()
            if objective_function_with_constraints_g2(parent1) > objective_function_with_constraints_g2(parent2):
                child1_h = parent1 + r * (parent1 - parent2)
                child2_h = parent2 + r * (parent2 - parent1)
            else:
                child1_h = parent2 + r * (parent2 - parent1)
                child2_h = parent1 + r * (parent1 - parent2)
            child1_h, child2_h = np.clip(child1_h, lb, ub), np.clip(child2_h, lb, ub)

            # Linear Crossover
            r = np.random.rand()
            child1_l = r * parent1 + (1 - r) * parent2
            child2_l = r * parent2 + (1 - r) * parent1
            child1_l, child2_l = np.clip(child1_l, lb, ub), np.clip(child2_l, lb, ub)

            children = [child1, child2, child1_u, child2_u, child1_h, child2_h, child1_l, child2_l]
            for child in children:
                if constraints_g2(child):  # Only add child if it satisfies the constraints
                    if not any(np.array_equal(child, x) for x in population):  # Check if the child already exists in the population
                        offspring.append(child)


  

    # Substituição dos piores indivíduos pelos filhos
    tamanho_substituicao = min(len(offspring), len(population))
    indices_substituicao = np.random.choice(len(population), size=tamanho_substituicao, replace=False)
    for i, index in enumerate(indices_substituicao):
        population[index] = offspring[i]

    # Mutação uniforme
    best_index = np.argmin(fitness_values)  # Encontre o índice do melhor indivíduo
    for i in range(len(population)):
        if i != best_index and np.random.rand() <= mutation_prob:  # Não mutar o melhor indivíduo
            mutation_point = np.random.randint(genes)
            new_value = np.random.uniform(lb[mutation_point], ub[mutation_point])
            temp_individual = population[i].copy()
            temp_individual[mutation_point] = new_value
            if constraints_g2(temp_individual):  # Aceite a mutação apenas se o indivíduo mutado satisfizer as restrições
                population[i] = temp_individual

    # Avaliação da população
    fitness_values = [objective_function_with_constraints_g2(ind) for ind in population]



    # Estatísticas da geração
    fitness_values = [objective_function_with_constraints_g2(ind) for ind in population]
    best_fitness = min(fitness_values)
    worst_fitness = max(fitness_values)
    mean_fitness = np.mean(fitness_values)
    std_dev_fitness = np.std(fitness_values)

    results_ga.append({
        'geracao': gen,
        'melhor_fitness': best_fitness,
        'melhor_particula': population[np.argmin(fitness_values)],
        'pior_fitness': worst_fitness,
        'media_dos_fitness': mean_fitness,
        'desvio_padrao': std_dev_fitness,
        'n_de_individuos': len(population),
    })

df_ga = pd.DataFrame(results_ga)
df_ga.head(5)

<ipython-input-28-57527fe5af41>:58: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  mask = np.random.randint(0, 2, size=genes).astype(np.bool)


,geracao,melhor_fitness,melhor_particula,pior_fitness,media_dos_fitness,desvio_padrao,n_de_individuos
0,0,[1211.6463778252532],"[1.8896313623594905, 2.2446126564660833, -0.73...",[10002649.172009312],659512.866614,1.993487e+06,50
1,1,[1145.312915925347],"[1.8896313541392107, 2.2446126496675207, -0.73...",[879250.8955527523],26889.671053,1.246662e+05,50
2,2,[1022.1590213744955],"[1.889631365859412, 2.244612657272547, -0.7342...",[8035414.076913635],247359.457637,1.260776e+06,50
3,3,[966.4153868009536],"[1.8896313607458788, 2.2446126568608507, -0.73...",[7023.954454332906],1924.131863,1.119052e+03,50
4,4,[1055.3904348742353],"[1.8896313565884113, 2.2446126541293774, -0.73...",[4530.586279985111],1454.688739,5.602403e+02,50


In [29]:
df_ga = df_ga.sort_values('melhor_fitness', ascending=True).reset_index()
best_ga_individual = df_ga.loc[0, 'melhor_particula']
fitness_best_ga_individual = df_ga.loc[0, 'melhor_fitness']
df_ga.head()

,index,geracao,melhor_fitness,melhor_particula,pior_fitness,media_dos_fitness,desvio_padrao,n_de_individuos
0,498,498,[697.5849740468416],"[1.88963136010605, 2.2446126556962462, -0.7342...",[6176.191344590634],807.501645,766.959461,50
1,497,497,[697.5849839567034],"[1.8896313533974827, 2.2446126557820207, -0.73...",[1149.420898436958],708.840473,64.827249,50
2,499,499,[697.5849857093446],"[1.8896313309106139, 2.244612653021962, -0.734...",[4949492.205359653],101438.969186,692667.916824,50
3,496,496,[697.5849885636608],"[1.889631343626981, 2.2446126535145074, -0.734...",[1535.0764412370793],714.334826,117.248802,50
4,495,495,[697.5849930021669],"[1.889631361219588, 2.2446126524729633, -0.734...",[702169.8233967872],14727.029767,98206.113376,50


## DE 

In [31]:
# Definindo os parâmetros do de
pop_size = 50
genes = 7
generations = 500
lb = [-10]*7
ub = [10]*7
crossover_prob = 0.6
mutation_prob = 0.2
F = 0.8  # Fator de escala para a mutação

results_de = []


# Inicialização da população
particles = np.empty((n_particles, n_dimensions))
for i in range(n_particles):
    while True:
        particle = np.random.uniform(low=lb, high=ub, size=n_dimensions)
        if objective_function_with_constraints_g2(particle) != 150000:
            particles[i] = particle
            break
population = particles

# Loop das gerações
for gen in range(generations):
    # Avaliação da população
    fitness_values = [objective_function_with_constraints_g2(ind) for ind in population]

    # Salvar o melhor indivíduo para elitismo
    best_index = np.argmin(fitness_values)
    best_individual = population[best_index].copy()

    # Evolução Diferencial
    new_population = population.copy()
    for i in range(pop_size):
        indices = [idx for idx in range(pop_size) if idx != i]
        a, b, c = population[np.random.choice(indices, 3, replace=False)]
        mutant = np.clip(a + F * (b - c), lb, ub)  # Mutação
        cross_points = np.random.rand(genes) < crossover_prob
        if not np.any(cross_points):
            cross_points[np.random.randint(0, genes)] = True
        trial = np.where(cross_points, mutant, population[i])  # Crossover
        if objective_function_with_constraints_g2(trial) < fitness_values[i]:  # Aceitação
            new_population[i] = trial

    population = new_population

    # Elitismo: garantir que o melhor indivíduo esteja na nova população
    new_fitness_values = [objective_function_with_constraints_g2(ind) for ind in population]
    worst_index = np.argmax(new_fitness_values)
    if new_fitness_values[worst_index] > fitness_values[best_index]:
        population[worst_index] = best_individual

    # Estatísticas da geração
    fitness_values = [objective_function_with_constraints_g2(ind) for ind in population]
    best_fitness = min(fitness_values)
    worst_fitness = max(fitness_values)
    mean_fitness = np.mean(fitness_values)
    std_dev_fitness = np.std(fitness_values)

    results_de.append({
        'geracao': gen,
        'melhor_fitness': best_fitness,
        'melhor_particula': population[np.argmin(fitness_values)],
        'pior_fitness': worst_fitness,
        'media_dos_fitness': mean_fitness,
        'desvio_padrao': std_dev_fitness,
        'n_de_individuos': len(population),
    })

df_de = pd.DataFrame(results_de)
df_de.head(5)

,geracao,melhor_fitness,melhor_particula,pior_fitness,media_dos_fitness,desvio_padrao,n_de_individuos
0,0,[1474.233534218936],"[-2.166067187114913, 0.7108044541917344, 0.092...",[1840699.1434510862],[175900.50287077442],[333076.9642325594],50
1,1,[1474.233534218936],"[-2.166067187114913, 0.7108044541917344, 0.092...",[388118.9709370994],[80861.23754999625],[84596.69492494097],50
2,2,[1474.233534218936],"[-2.166067187114913, 0.7108044541917344, 0.092...",150000,[65853.29492954223],[67548.67036699633],50
3,3,[1474.233534218936],"[-2.166067187114913, 0.7108044541917344, 0.092...",150000,[61822.3900904528],[67410.55194493997],50
4,4,[1474.233534218936],"[-2.166067187114913, 0.7108044541917344, 0.092...",150000,[55379.05912332384],[65984.62722065159],50


In [32]:
df_de = df_de.sort_values('melhor_fitness', ascending=True).reset_index()
best_de_individual = df_de.loc[0, 'melhor_particula']
fitness_best_de_individual = df_de.loc[0, 'melhor_fitness']
df_de.head()

,index,geracao,melhor_fitness,melhor_particula,pior_fitness,media_dos_fitness,desvio_padrao,n_de_individuos
0,499,499,[680.6712807856551],"[2.364039909893986, 1.9579673568476899, -0.402...",[680.6712807856551],680.671281,0.0,50
1,401,401,[680.6712807856551],"[2.364039909893986, 1.9579673568476899, -0.402...",[680.6712807856551],680.671281,0.0,50
2,400,400,[680.6712807856551],"[2.364039909893986, 1.9579673568476899, -0.402...",[680.6712807856551],680.671281,0.0,50
3,399,399,[680.6712807856551],"[2.364039909893986, 1.9579673568476899, -0.402...",[680.6712807856551],680.671281,0.0,50
4,398,398,[680.6712807856551],"[2.364039909893986, 1.9579673568476899, -0.402...",[680.6712807856551],680.671281,0.0,50


##ES

In [33]:
## # ES 

# Definindo os parâmetros da evolução estratégica
pop_size = 50
genes = 7
generations = 500
lb = [-10]*7
ub = [10]*7
sigma = 0.01  # Desvio padrão para a mutação
tau = 1/np.sqrt(2*genes)  # Parâmetro de autoadaptação

results_es = []
np.random.seed(24)

# Inicialização da população
particles = np.empty((n_particles, n_dimensions))
for i in range(n_particles):
    while True:
        particle = np.random.uniform(low=lb, high=ub, size=n_dimensions)
        if objective_function_with_constraints_g2(particle) != 150000:
            particles[i] = particle
            break
population = particles
sigmas = sigma * np.ones_like(population)

# Loop das gerações
for gen in range(generations):
    # Avaliação da população
    fitness_values = [objective_function_with_constraints_g2(ind) for ind in population]

    # Estratégia Evolutiva
    for i in range(pop_size):
        child = population[i].copy()
        child_sigma = sigmas[i].copy()
        child_sigma *= np.exp(tau * np.random.randn())  # Autoadaptação
        child += child_sigma * np.random.randn(genes)  # Mutação
        child = np.clip(child, lb, ub)
        if objective_function_with_constraints_g2(child) < fitness_values[i]:  # Seleção
            population[i] = child
            sigmas[i] = child_sigma

    # Estatísticas da geração
    fitness_values = [objective_function_with_constraints_g2(ind) for ind in population]
    best_fitness = min(fitness_values)
    worst_fitness = max(fitness_values)
    mean_fitness = np.mean(fitness_values)
    std_dev_fitness = np.std(fitness_values)

    results_es.append({
        'geracao': gen,
        'melhor_fitness': best_fitness,
        'melhor_particula': population[np.argmin(fitness_values)],
        'pior_fitness': worst_fitness,
        'media_dos_fitness': mean_fitness,
        'desvio_padrao': std_dev_fitness,
        'n_de_individuos': len(population),
    })

df_es = pd.DataFrame(results_es)
df_es.head(5)

/usr/local/lib/python3.10/dist-packages/numpy/core/_methods.py:163: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  arr = asanyarray(a)
/usr/local/lib/python3.10/dist-packages/numpy/core/_methods.py:197: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  arr = asanyarray(a)
/usr/local/lib/python3.10/dist-packages/numpy/core/fromnumeric.py:43: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must sp

,geracao,melhor_fitness,melhor_particula,pior_fitness,media_dos_fitness,desvio_padrao,n_de_individuos
0,0,[1899.4487466089147],"[-0.6476212747711542, 0.7924334409512752, 1.64...",[9529442.33836956],1619789.800384,2325071.460791,50
1,1,[1899.0725865719164],"[-0.6476212747711542, 0.7924334409512752, 1.64...",[9526337.334223196],[1532278.1453317055],[2292756.7917898777],50
2,2,[1899.0725865719164],"[-0.6476212747711542, 0.7924334409512752, 1.64...",[9463968.445088677],[1528491.9203807816],[2285898.155499782],50
3,3,[1892.96670502713],"[-0.6476212747711542, 0.7924334409512752, 1.64...",[9410099.117888335],[1522878.0720364545],[2278860.891616909],50
4,4,[1892.96670502713],"[-0.6476212747711542, 0.7924334409512752, 1.64...",[9214793.818274237],[1516144.8341334134],[2262907.5619541644],50


In [34]:
df_es = df_es.sort_values('melhor_fitness', ascending=True).reset_index()
best_es_individual = df_es.loc[0, 'melhor_particula']
fitness_best_es_individual = df_es.loc[0, 'melhor_fitness']
df_es.head()

,index,geracao,melhor_fitness,melhor_particula,pior_fitness,media_dos_fitness,desvio_padrao,n_de_individuos
0,499,499,[693.0545641671271],"[2.3974164849209862, 1.9919566597758598, -0.61...",[1539292.6077232964],[174412.29018933207],[375645.3065167732],50
1,465,465,[693.0545641671271],"[2.3974164849209862, 1.9919566597758598, -0.61...",[1544558.2759442953],[189239.55522705923],[385285.97016863484],50
2,464,464,[693.0545641671271],"[2.3974164849209862, 1.9919566597758598, -0.61...",[1544558.2759442953],[189514.67778474453],[385699.7258628399],50
3,463,463,[693.0545641671271],"[2.3974164849209862, 1.9919566597758598, -0.61...",[1544558.2759442953],[190142.22597111552],[386813.0267156861],50
4,462,462,[693.0545641671271],"[2.3974164849209862, 1.9919566597758598, -0.61...",[1544562.486657384],[190160.43519061507],[386826.3522584588],50


## resultados obtidos

In [35]:
resultados_G2 = pd.DataFrame({
    'EVO. Alg.':['PSO','GA','DE','ES'],
    'Best Fitness':[fitness_best_PSO_individual,fitness_best_ga_individual,fitness_best_de_individual,fitness_best_es_individual],
    'Best Particle EA':[best_PSO_individual,best_ga_individual,best_de_individual,best_es_individual]})

resultados_G2 = resultados_G2.sort_values('Best Fitness',ascending=True).reset_index()
resultados_G2.head()

,index,EVO. Alg.,Best Fitness,Best Particle EA
0,2,DE,[680.6712807856551],"[2.364039909893986, 1.9579673568476899, -0.402..."
1,0,PSO,[680.7384017684655],"[2.31892106209504, 1.9639805917639315, -0.4730..."
2,3,ES,[693.0545641671271],"[2.3974164849209862, 1.9919566597758598, -0.61..."
3,1,GA,[697.5849740468416],"[1.88963136010605, 2.2446126556962462, -0.7342..."
